In [1]:
import itertools
from multiprocessing import cpu_count, Pool
import pickle
from collections import Counter
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torchtext import data
from nltk.tokenize import word_tokenize
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm
from joblib import Parallel, delayed

In [2]:
with open('../Sentiment-and-Style-Transfer/data/yelp/sentiment.train.0', 'rt') as in_f:
    sentences = in_f.readlines()
with open('../Sentiment-and-Style-Transfer/data/yelp/sentiment.train.1', 'rt') as in_f:
    sentences.extend(in_f.readlines())

In [3]:
df = pd.DataFrame({0: sentences})
df = df.sample(frac=1.0)

In [4]:
with open('../Sentiment-and-Style-Transfer/data/yelp/sentiment.train', 'wt') as out_f:
    for sen in list(df[0]):
        out_f.write(sen)

In [6]:
# train model
model = Word2Vec(corpus_file='../Sentiment-and-Style-Transfer/data/yelp/sentiment.train', min_count=5, workers=8, window=5, size=300, sg=1)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
print('Vocab size: %s' % len(words))

# save model
model.save('output/wv.yelp.bin')

Word2Vec(vocab=9349, size=300, alpha=0.025)
Vocab size: 9349


In [ ]:
new_model = Word2Vec.load('output/wv.bin')
print(new_model)

In [ ]:
def export_wv(path_to_file, wv):
    vocabs = list(wv.vocab.keys())
    vector_size = wv[vocabs[0]].shape[0]
    with open(path_to_file, 'wt') as out:
        out.write('{} {}\n'.format(len(vocabs), vector_size))
        for vocab in tqdm(vocabs):
            vec = ' '.join([str(item) for item in wv[vocab]])
            out.write('{} {}\n'.format(vocab, vec))
            
wv = new_model.wv
export_wv('./output/wv.vec', wv)